## Pandas Merging Two Data Frames Using Group By Count Function
### (Similar to using ROW_NUMBER() Windows Function in MS SQL/T-SQL)
#### Programmer: Vincent Sanchez
#### Date: 2022-02-02
#### Description: This notebook walks through the process of importing two excel workbooks: one containing monthly budget data and the other annual budget data to join them together to create one dataset where annual budget data is availabe to distinct groups.

#### Context: Stakeholder wanted to both monthly and annual budget data from an excel workbook to be visualized in Power BI. Aggregation would be done at both the monthly and annual variances to the cost center/charging expense type level. Data would then be joined to another dataset that is using an Oracle database.

#### Instructions: Use the files provied in the example to run through the program to understand the logic. Update the cells to fit your needs.

***
*Note: In each cell, there is a reference to the dataframe being used. Simply remove that line if you do not want to view the transformation of that dataframe*

In [109]:
# Import pandas framework
import pandas as pd

In [110]:
# Update paths to the path where the workbooks are hosted.
path_monthly = 'C:/Users/USERNAME/Documents/example/monthly.xlsx' # In Git-Hub repo
path_annual = 'C:/Users/USERNAME/Documents/example/annual.xlsx' # In Git-Hub repo
path_merged = 'C:/Users/USERNAME/Documents/example/merged.xlsx' # New File

In [111]:
# Import two Excel Workbooks into two separate dataframes (df_monthly for monthly budget and df_annual for annual budget)
df_monthly = pd.read_excel(path_monthly)
df_annual = pd.read_excel(path_annual)

In [112]:
# Sort df_annual data frame by columns to be counted over, ascending.
df_annual = df_annual.sort_values(by = ['Month','Year','Cost Center','Charging Expense Type'], ascending = True)
df_annual

,Month,Year,Cost Center,Charging Expense Type,Amount
6,TOTAL,2021,A,A,30303030
7,TOTAL,2021,A,B,9999111
8,TOTAL,2021,B,A,10000
9,TOTAL,2021,B,B,107064
10,TOTAL,2021,B,C,986884
11,TOTAL,2021,C,B,187846
0,TOTAL,2022,A,A,1000000
1,TOTAL,2022,A,B,2000000
2,TOTAL,2022,B,A,300000000
3,TOTAL,2022,B,B,30000


In [113]:
# Sort df_monthly data frame by columns to be counted over, ascending.
df_monthly = df_monthly.sort_values(by = ['Year','Cost Center','Charging Expense Type'], ascending = True)
df_monthly

,Month,Year,Cost Center,Charging Expense Type,Amount
50,1,2021,A,A,100
51,2,2021,A,A,101
52,3,2021,A,A,102
53,4,2021,A,A,103
54,5,2021,A,A,104
...,...,...,...,...,...
45,8,2022,C,B,11111
46,9,2022,C,B,200000
47,10,2022,C,B,10000
48,11,2022,C,B,1000


In [114]:
# Create column 'row_count' over sorted columns and create a cummulative count.
df_annual['row_count'] = df_annual.groupby(['Year','Cost Center','Charging Expense Type']).cumcount()+1 
df_annual

,Month,Year,Cost Center,Charging Expense Type,Amount,row_count
6,TOTAL,2021,A,A,30303030,1
7,TOTAL,2021,A,B,9999111,1
8,TOTAL,2021,B,A,10000,1
9,TOTAL,2021,B,B,107064,1
10,TOTAL,2021,B,C,986884,1
11,TOTAL,2021,C,B,187846,1
0,TOTAL,2022,A,A,1000000,1
1,TOTAL,2022,A,B,2000000,1
2,TOTAL,2022,B,A,300000000,1
3,TOTAL,2022,B,B,30000,1


In [115]:
# Create column 'row_count' over sorted columns and create a cummulative count.
df_monthly['row_count'] = df_monthly.groupby(['Year','Cost Center','Charging Expense Type']).cumcount()+1 
df_monthly

,Month,Year,Cost Center,Charging Expense Type,Amount,row_count
50,1,2021,A,A,100,1
51,2,2021,A,A,101,2
52,3,2021,A,A,102,3
53,4,2021,A,A,103,4
54,5,2021,A,A,104,5
...,...,...,...,...,...,...
45,8,2022,C,B,11111,1
46,9,2022,C,B,200000,2
47,10,2022,C,B,10000,3
48,11,2022,C,B,1000,4


In [116]:
# Merge df_annual and df_monthly to columns via a left join on Year, Cost Center and Charging Expense Type.
df_combined = pd.merge(df_monthly, df_annual, how='left', left_on=['Year','Cost Center','Charging Expense Type','row_count'], right_on=['Year','Cost Center','Charging Expense Type','row_count'])
df_combined

,Month_x,Year,Cost Center,Charging Expense Type,Amount_x,row_count,Month_y,Amount_y
0,1,2021,A,A,100,1,TOTAL,30303030.0
1,2,2021,A,A,101,2,NaN,NaN
2,3,2021,A,A,102,3,NaN,NaN
3,4,2021,A,A,103,4,NaN,NaN
4,5,2021,A,A,104,5,NaN,NaN
...,...,...,...,...,...,...,...,...
95,8,2022,C,B,11111,1,TOTAL,10000.0
96,9,2022,C,B,200000,2,NaN,NaN
97,10,2022,C,B,10000,3,NaN,NaN
98,11,2022,C,B,1000,4,NaN,NaN


In [117]:
# Rename columns.
df_combined.rename(columns={'Month_x':'Month','Year_x':'Year','Amount_x':'Monthly Amount','Amount_y':'Annual Budget'}, inplace=True)
df_combined

,Month,Year,Cost Center,Charging Expense Type,Monthly Amount,row_count,Month_y,Annual Budget
0,1,2021,A,A,100,1,TOTAL,30303030.0
1,2,2021,A,A,101,2,NaN,NaN
2,3,2021,A,A,102,3,NaN,NaN
3,4,2021,A,A,103,4,NaN,NaN
4,5,2021,A,A,104,5,NaN,NaN
...,...,...,...,...,...,...,...,...
95,8,2022,C,B,11111,1,TOTAL,10000.0
96,9,2022,C,B,200000,2,NaN,NaN
97,10,2022,C,B,10000,3,NaN,NaN
98,11,2022,C,B,1000,4,NaN,NaN


In [118]:
# Temporarily drop unecessary columns until needed.
df_combined.drop(['row_count','Month_y'], inplace=True, axis=1)
df_combined

,Month,Year,Cost Center,Charging Expense Type,Monthly Amount,Annual Budget
0,1,2021,A,A,100,30303030.0
1,2,2021,A,A,101,NaN
2,3,2021,A,A,102,NaN
3,4,2021,A,A,103,NaN
4,5,2021,A,A,104,NaN
...,...,...,...,...,...,...
95,8,2022,C,B,11111,10000.0
96,9,2022,C,B,200000,NaN
97,10,2022,C,B,10000,NaN
98,11,2022,C,B,1000,NaN


In [119]:
# Uppdates Annual Budget NaN type to 0
df_combined['Annual Budget'].fillna(0, inplace=True)
df_combined

,Month,Year,Cost Center,Charging Expense Type,Monthly Amount,Annual Budget
0,1,2021,A,A,100,30303030.0
1,2,2021,A,A,101,0.0
2,3,2021,A,A,102,0.0
3,4,2021,A,A,103,0.0
4,5,2021,A,A,104,0.0
...,...,...,...,...,...,...
95,8,2022,C,B,11111,10000.0
96,9,2022,C,B,200000,0.0
97,10,2022,C,B,10000,0.0
98,11,2022,C,B,1000,0.0


In [120]:
# Write to file.
df_combined.to_excel(path_merged, index=False, sheet_name='budget_merged_annual_budget')

## `End Notebook`